In [1]:
from abc import ABC, abstractmethod
from pathlib import Path
from typing import List
from functools import partial, reduce

import pandas as pd
import numpy as np
import matplotlib.pyplot as plt

In [2]:
DATA_CLEANED_DIR = Path('../data/cleaned/')

In [3]:
# import files
df_country_codes = pd.read_csv(Path(DATA_CLEANED_DIR, 'country_codes.csv'))
df_life_exp = pd.read_csv(Path(DATA_CLEANED_DIR, 'life_expectancy.csv'))
df_mortality = pd.read_csv(Path(DATA_CLEANED_DIR, 'mortality.csv'))
df_population = pd.read_csv(Path(DATA_CLEANED_DIR, 'population.csv'))

C:\Users\ddlat\anaconda3\lib\site-packages\IPython\core\interactiveshell.py:3444: DtypeWarning: Columns (2) have mixed types.Specify dtype option on import or set low_memory=False.
  exec(code_obj, self.user_global_ns, self.user_ns)


# Netherlands & Japan & Canada
We first want to visualize the data of The Netherlands

In [4]:
df_country_codes.head(1)
df_country_codes[(df_country_codes['country'] == 'Netherlands') | 
                 (df_country_codes['country'] == 'Japan') |
                 (df_country_codes['country'] == 'Canada')]

,country code,country
69,2090,Canada
124,3160,Japan
186,4210,Netherlands


As we can see, the country code of The Netherlands is `4210`. Therefore we will filter the *life expectancy*, *mortality* and *population datasets on this country code.

In [5]:
NL_CODE = 4210
JP_CODE = 3160
CA_CODE = 2090

At the moment the **Life expectancy** dataset contains only data for the Netherlands. The data is obtained from https://www.who.int/data/maternal-newborn-child-adolescent-ageing/indicator-explorer-new/mca/life-expectancy-at-birth. Due to the fact that the *Export* button did not work at that time, I copied it manually in the dataset. 

In [6]:
df_life_exp

,year,country,sex,life expectancy [age]
0,1950,Netherlands,3,71.411
1,1950,Netherlands,2,72.615
2,1951,Netherlands,3,71.550
3,1950,Netherlands,1,70.236
4,1951,Netherlands,2,72.775
...,...,...,...,...
583,2018,Canada,3,82.315
584,2018,Canada,2,84.308
585,2017,Canada,3,82.210
586,2017,Canada,2,84.232


In [7]:
nl_life_exp = df_life_exp[(df_life_exp['country'] == 'Netherlands')]
jp_life_exp = df_life_exp[(df_life_exp['country'] == 'Japan')]
ca_life_exp = df_life_exp[(df_life_exp['country'] == 'Canada')]


nl_mortality = df_mortality[(df_mortality['country code'] == NL_CODE)]
jp_mortality = df_mortality[(df_mortality['country code'] == JP_CODE)]
ca_mortality = df_mortality[(df_mortality['country code'] == CA_CODE)]

nl_population = df_population[(df_population['country code'] == NL_CODE)]
jp_population = df_population[(df_population['country code'] == JP_CODE)]
ca_population = df_population[(df_population['country code'] == CA_CODE)]

So it is best to merge them into one single file. In this way we can standardize the values. You want to merge them on `year` and `sex`.

Coding: 
`{1: male, 2: female, 3: both sexes}`

In [213]:
life_exp_set = [nl_life_exp, jp_life_exp, ca_life_exp]
mort_set = [nl_mortality, jp_mortality, ca_mortality]
pop_set = [nl_population, jp_population, ca_population]

In [214]:
for df in life_exp_set:
    print(df.sex.unique())

[3 2 1]
[3 2 1]
[3 2 1]


In [215]:
for df in mort_set:
    print(df.sex.unique())

[1 2]
[1 2]
[1 2 9]


In [216]:
for df in pop_set:
    print(df.sex.unique())

[1 2]
[1 2]
[1 2 9]


In [217]:
ca_mortality[ca_mortality['sex'] == 9]

,country code,year,list,cause,sex,deaths
209150,2090,2000,104,AAA,9,1
209151,2090,2000,104,P011,9,1
212946,2090,2001,104,AAA,9,1
212947,2090,2001,104,Q336,9,1


In [218]:
ca_population[ca_population['sex'] == 9]

,country code,year,sex,population
854,2090,1999,9,0.0


In [219]:
# drop sex 9 as this is not usesful
ca_mortality = ca_mortality[~ca_mortality['sex'].isin([9])]
ca_population = ca_population[~ca_population['sex'].isin([9])]

#### It is first important to create an aggregate of sexes for the *mortality* and *population* dataset.

In [220]:
def generate_ICD_codes(lower, upper, symbol):
    codes = []
    for i in range(lower, upper + 1, 1):
        if i < 10:
            codes.append(f'{symbol}0{i}')
        else:
            codes.append(f'{symbol}{i}')

    return np.array(codes)


def convert_format(series, n=3):
    """Only keep the n first characters of the column"""
    return series.apply(lambda x: x[:n])


def filter_column(df: pd.DataFrame, column: str, elements):
    """


    :param df:
    :param column:
    :param elements:
    :return:
    """
    target = df[column].unique()
    found_elements = find_elements(target, elements)
    dataset = df[df[column].isin(found_elements)]

    return dataset


def groupby_sum(df, by, on):
    """Groups the dataframe by the index columns, and sums the target column and
    returns the result as a dataframe."""

    grouped = df.groupby(by, as_index=False)[on].sum()

    return grouped


def find_elements(target, elements):
    """
    Checks if the targets can be found in an arbitrary list of elements.

    :param elements:
    :param target:
    :return:
    """
    mask = np.isin(target, elements)
    found = np.where(mask, target, '')
    valid = [c for c in found if c != '']

    return valid

In [273]:
class Selector(ABC):

    def __init__(self, file):
        self.file = file

    @abstractmethod
    def get_selection(self):
        pass


class DataFrameSelector(Selector):

    def __init__(self, df):
        if not isinstance(df, pd.DataFrame):
            raise ValueError('Expects a pandas DataFrame.')
        super().__init__(df)

        self.selection = {}

    def filter_column(self, column: str, elements):
        target = self.file[column].unique()
        found_elements = find_elements(target, elements)
        dataset = self.file[self.file[column].isin(found_elements)]
        
        return dataset

    def split_dataframe(self, column: str, labels: List[str], selection: List[np.ndarray]):
        """Expects labels as keys and the selection to be the string to select the dataframe on.

        if unique elements are given then it searches for elements found in the selection and the unique list
        """

        for label, selector in zip(labels, selection):
            dataset = self.filter_column(column, selector)
            self.selection[label] = dataset
            
    def rename_selection(self, column, mapping):
        for k, df in self.selection.items():
            self.selection[k] = df.rename(columns={column: mapping[k]})

    def get_selection(self):
        return self.selection

In [222]:
class Aggregator:

    def __init__(self, df):
        self.df = df
        self.aggregation = None

    def handler(self, df):
        if self.aggregation is None:
            self.aggregation = df
        else:
            self.aggregation = self.aggregation.append(df)

    def calc_aggr(self, by, on, column='', value=None):
        # calculate aggregate and append it to the original
        if not column and not value:
            self.handler(groupby_sum(self.df, by, on))
            return

        aggr = groupby_sum(self.df, by, on)
        aggr[column] = value

        self.handler(aggr)

    def get_aggregation(self, sort_by):
        return self.aggregation.sort_values(sort_by).reset_index(drop=True)

In [223]:
C_codes = generate_ICD_codes(0, 97, 'C')
I_codes = generate_ICD_codes(5, 99, 'I')
E_codes = generate_ICD_codes(10, 13, 'E')
J_codes = generate_ICD_codes(40, 47, 'J')
K_codes = generate_ICD_codes(0, 93, 'K')

code_to_name_map = {
    'C': 'cancer [deaths]',
    'I': 'cardiovascular disease [deaths]',
    'E': 'diabetes mellitus [deaths]',
    'J': 'chronic respiratory diseases [deaths]',
    'K': 'diseases of digestive system [deaths]'
}

name_to_code_map = {
    'cancer [deaths]': 'C',
    'cardiovascular disease [deaths]': 'I',
    'diabetes mellitus [deaths]': 'E',
    'chronic respiratory diseases [deaths]': 'J',
    'diseases of digestive system [deaths]': 'K',
}

code_map = {
    'C': 'C',
    'I': 'I',
    'E': 'E',
    'J': 'J',
    'K': 'K'
}

codes = [C_codes, I_codes, E_codes, J_codes, K_codes]
labels = ['C', 'I', 'E', 'J', 'K']

In [224]:
def convert_format(items: list, n=3):
    """Only keep the n first characters of the column"""
    series = pd.Series(items)
    return series.apply(lambda x: x[:n])

In [264]:
nl_causes = convert_format(nl_mortality['cause'], 3)
jp_causes = convert_format(jp_mortality['cause'], 3)
ca_causes = convert_format(ca_mortality['cause'], 3)

nl_unique_causes = nl_causes.unique()
jp_unique_causes = jp_causes.unique()
ca_unique_causes = ca_causes.unique()

nl_mortality.loc[:, 'cause'] = nl_causes
jp_mortality.loc[:, 'cause'] = jp_causes
ca_mortality.loc[:, 'cause'] = ca_causes

C:\Users\ddlat\anaconda3\lib\site-packages\pandas\core\indexing.py:1773: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  self._setitem_single_column(ilocs[0], value, pi)


In [265]:
nl_mortality

,country code,year,list,cause,sex,deaths
797892,4210,1996,10M,A02,1,2
797893,4210,1996,10M,A02,1,3
797894,4210,1996,10M,A03,1,1
797895,4210,1996,10M,A04,1,1
797896,4210,1996,10M,A06,1,1
...,...,...,...,...,...,...
4197619,4210,2018,104,Y87,1,1
4197620,4210,2018,104,Y87,1,1
4197621,4210,2018,104,Y88,1,3
4197622,4210,2018,104,Y88,2,3


In [279]:
# Select correct data
nl_mortality_selector = DataFrameSelector(nl_mortality)
nl_mortality_selector.split_dataframe('cause', labels, codes)
nl_mortality_selector.rename_selection('deaths', code_to_name_map)
nl_mortality_sets = nl_mortality_selector.get_selection()

jp_mortality_selector = DataFrameSelector(jp_mortality)
jp_mortality_selector.split_dataframe('cause', labels, codes)
jp_mortality_selector.rename_selection('deaths', code_to_name_map)
jp_mortality_sets = jp_mortality_selector.get_selection()

ca_mortality_selector = DataFrameSelector(ca_mortality)
ca_mortality_selector.split_dataframe('cause', labels, codes)
ca_mortality_selector.rename_selection('deaths', code_to_name_map)
ca_mortality_sets = ca_mortality_selector.get_selection()

# create aggregates
for k, df in nl_mortality_sets.items():
    aggr = Aggregator(df)
    aggr.calc_aggr(by=['year', 'sex'], on=code_to_name_map[k])
    aggr.calc_aggr(by=['year'], on=code_to_name_map[k], column='sex', value=3)
    
    data = aggr.get_aggregation(sort_by='year')
    data['country'] = 'Netherlands' 
    
    nl_mortality_sets[k] = data
    
    
for k, df in jp_mortality_sets.items():
    aggr = Aggregator(df)
    aggr.calc_aggr(by=['year', 'sex'], on=code_to_name_map[k])
    aggr.calc_aggr(by=['year'], on=code_to_name_map[k], column='sex', value=3)
    
    data = aggr.get_aggregation(sort_by='year')
    data['country'] = 'Japan'
    
    jp_mortality_sets[k] = data
    
for k, df in ca_mortality_sets.items():
    aggr = Aggregator(df)
    aggr.calc_aggr(by=['year', 'sex'], on=code_to_name_map[k])
    aggr.calc_aggr(by=['year'], on=code_to_name_map[k], column='sex', value=3)
    
    data = aggr.get_aggregation(sort_by='year')
    data['country'] = 'Canada'
    
    ca_mortality_sets[k] = data

In [280]:
nl_mortality_sets['C'].head()

,year,sex,cancer [deaths],country
0,1996,1,20754,Netherlands
1,1996,2,16466,Netherlands
2,1996,3,37220,Netherlands
3,1997,1,20420,Netherlands
4,1997,2,16713,Netherlands


In [281]:
jp_mortality_sets['C'].head()

,year,sex,cancer [deaths],country
0,1995,1,159623,Japan
1,1995,2,103399,Japan
2,1995,3,263022,Japan
3,1996,1,164824,Japan
4,1996,2,106359,Japan


In [282]:
ca_mortality_sets['C'].head()

,year,sex,cancer [deaths],country
0,2000,1,33360,Canada
1,2000,2,29312,Canada
2,2000,3,62672,Canada
3,2001,1,34023,Canada
4,2001,2,29751,Canada


### Population 
Population is already properly formatted, thus no need to make a selection.


In [341]:
aggr = Aggregator(nl_population)
aggr.calc_aggr(by=['year', 'sex'], on='population')
aggr.calc_aggr(by=['year'], on='population', column='sex', value=3)
data = aggr.get_aggregation(sort_by='year')
data['country'] = 'Netherlands'

nl_pop_agg = data
nl_pop_agg.isna().sum()

year          0
sex           0
population    0
country       0
dtype: int64

In [342]:
aggr = Aggregator(jp_population)
aggr.calc_aggr(by=['year', 'sex'], on='population')
aggr.calc_aggr(by=['year'], on='population', column='sex', value=3)
data = aggr.get_aggregation(sort_by='year')
data['country'] = 'Japan'

jp_pop_agg = data
jp_pop_agg.isna().sum()

year          0
sex           0
population    0
country       0
dtype: int64

In [343]:
aggr = Aggregator(ca_population)
aggr.calc_aggr(by=['year', 'sex'], on='population')
aggr.calc_aggr(by=['year'], on='population', column='sex', value=3)
data = aggr.get_aggregation(sort_by='year')
data['country'] = 'Canada'

ca_pop_agg = data
ca_pop_agg.isna().sum()

year          0
sex           0
population    0
country       0
dtype: int64

## Combine all 

In [344]:
# nl_pop_agg = nl_pop_agg[nl_pop_agg['year'] >= 1996]
# nl_life_exp = nl_life_exp[(nl_life_exp['year'] >= 1996) & (nl_life_exp['year'] <= 2018)]

# jp_pop_agg = jp_pop_agg[jp_pop_agg['year'] >= 1996]
# jp_life_exp = jp_life_exp[(jp_life_exp['year'] >= 1996) & (jp_life_exp['year'] <= 2018)]

# ca_pop_agg = ca_pop_agg[ca_pop_agg['year'] >= 1996]
# ca_life_exp = ca_life_exp[(ca_life_exp['year'] >= 1996) & (ca_life_exp['year'] <= 2018)]

In [345]:
nl_pop_agg.isna().sum()

year          0
sex           0
population    0
country       0
dtype: int64

In [324]:
jp_pop_agg.isna().sum()

year          0
sex           0
population    0
dtype: int64

In [325]:
ca_pop_agg.isna().sum()

year          0
sex           0
population    0
dtype: int64

In [366]:
nl_dfs = []
jp_dfs = []
ca_dfs = []

nl_dfs.extend([df for df in nl_mortality_sets.values()])
jp_dfs.extend([df for df in jp_mortality_sets.values()])
ca_dfs.extend([df for df in ca_mortality_sets.values()])

nl_dfs.append(nl_pop_agg)
nl_dfs.append(nl_life_exp)
jp_dfs.append(jp_pop_agg)
jp_dfs.append(jp_life_exp)
ca_dfs.append(ca_pop_agg)
ca_dfs.append(ca_life_exp)

In [362]:
def multi_merge(dfs, on):
    merge = partial(pd.merge, on=on, how='outer')
    dataset = reduce(merge, dfs)
    
    return dataset

In [378]:
nl_dataset = multi_merge(nl_dfs, on=['year', 'sex', 'country']).sort_values('year')
nl_dataset = dataset[(dataset['year'] > 1995) & (dataset['year'] < 2019)]

nl_dataset['non-communicable chronic disease [deaths]'] = 0
for disease in name_to_code_map.keys():
    nl_dataset['non-communicable chronic disease [deaths]'] += dataset[disease]
    
nl_dataset = nl_dataset.sort_values(['year', 'sex']).reset_index(drop=True)

In [379]:
jp_dataset = multi_merge(jp_dfs, on=['year', 'sex', 'country']).sort_values('year')
jp_dataset = dataset[(dataset['year'] > 1995) & (dataset['year'] < 2019)]

jp_dataset['non-communicable chronic disease [deaths]'] = 0
for disease in name_to_code_map.keys():
    jp_dataset['non-communicable chronic disease [deaths]'] += dataset[disease]
    
jp_dataset = jp_dataset.sort_values(['year', 'sex']).reset_index(drop=True)

In [380]:
ca_dataset = multi_merge(ca_dfs, on=['year', 'sex', 'country']).sort_values('year')
ca_dataset = dataset[(dataset['year'] > 1995) & (dataset['year'] < 2019)]

ca_dataset['non-communicable chronic disease [deaths]'] = 0
for disease in name_to_code_map.keys():
    ca_dataset['non-communicable chronic disease [deaths]'] += dataset[disease]
    
ca_dataset = ca_dataset.sort_values(['year', 'sex']).reset_index(drop=True)

In [397]:
full_dataset = pd.concat([nl_dataset, jp_dataset, ca_dataset]).reset_index(drop=True)
full_dataset

,year,sex,cancer [deaths],country,cardiovascular disease [deaths],diabetes mellitus [deaths],chronic respiratory diseases [deaths],diseases of digestive system [deaths],population,life expectancy [age],non-communicable chronic disease [deaths]
0,1996,1,20754.0,Netherlands,25205.0,382.0,4322.0,2206.0,7679500.0,74.783,52869.0
1,1996,2,16466.0,Netherlands,26102.0,514.0,2166.0,2951.0,7851000.0,80.378,48199.0
2,1996,3,37220.0,Netherlands,51307.0,896.0,6488.0,5157.0,15530500.0,77.642,101068.0
3,1997,1,20420.0,Netherlands,24309.0,339.0,4096.0,2128.0,7718400.0,74.963,51292.0
4,1997,2,16713.0,Netherlands,25448.0,561.0,2260.0,2883.0,7892200.0,80.435,47865.0
...,...,...,...,...,...,...,...,...,...,...,...
202,2017,2,20361.0,Netherlands,20052.0,474.0,3495.0,2385.0,8632105.0,83.724,46767.0
203,2017,3,44908.0,Netherlands,38145.0,928.0,7016.0,4475.0,17133498.0,82.004,95472.0
204,2018,1,24363.0,Netherlands,18264.0,475.0,3513.0,2258.0,8597564.0,80.419,48873.0
205,2018,2,20407.0,Netherlands,19520.0,463.0,3572.0,2465.0,8718843.0,83.838,46427.0


In [ ]:
full_dataset.to_csv('../data/prepared/full_dataset.csv', index=False)

In [398]:
cols = ['cancer [deaths]',
       'cardiovascular disease [deaths]', 'diabetes mellitus [deaths]',
       'chronic respiratory diseases [deaths]',
       'diseases of digestive system [deaths]',
       'non-communicable chronic disease [deaths]']
full_dataset_standardized = full_dataset
full_dataset_standardized[cols] = full_dataset[cols].divide(full_dataset['population'], axis=0) * 100000

full_dataset_standardized

,year,sex,cancer [deaths],country,cardiovascular disease [deaths],diabetes mellitus [deaths],chronic respiratory diseases [deaths],diseases of digestive system [deaths],population,life expectancy [age],non-communicable chronic disease [deaths]
0,1996,1,270.251970,Netherlands,328.211472,4.974282,56.279706,28.725829,7679500.0,74.783,688.443258
1,1996,2,209.731244,Netherlands,332.467202,6.546937,27.588842,37.587568,7851000.0,80.378,613.921793
2,1996,3,239.657448,Netherlands,330.362834,5.769293,41.775860,33.205628,15530500.0,77.642,650.771063
3,1997,1,264.562604,Netherlands,314.948694,4.392102,53.067993,27.570481,7718400.0,74.963,664.541874
4,1997,2,211.766047,Netherlands,322.444946,7.108284,28.635868,36.529738,7892200.0,80.435,606.484884
...,...,...,...,...,...,...,...,...,...,...,...
202,2017,2,235.875259,Netherlands,232.295599,5.491129,40.488386,27.629414,8632105.0,83.724,541.779786
203,2017,3,262.106430,Netherlands,222.634047,5.416290,40.949023,26.118426,17133498.0,82.004,557.224217
204,2018,1,283.370964,Netherlands,212.432266,5.524821,40.860411,26.263253,8597564.0,80.419,568.451715
205,2018,2,234.056285,Netherlands,223.882917,5.310338,40.968739,28.272100,8718843.0,83.838,532.490377


In [394]:

full_dataset_standardized.to_csv('../data/prepared/full_dataset_standardized.csv', index=False)